In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

C:\Users\NEPC LAPTOP3\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\NEPC LAPTOP3\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
df= pd.read_csv("./Machine Learning.csv")
df.head()

,Name,CAT,Month Test Taken,Sortie Type,Overall Grade,Mission Items,Original comments,Specific comments,Sentiment Score,Normalised Grades,Sentiment Term
0,A ONG,P,2/10/2018,EPP,4,Dual DMU Fail,Actions as required.,actions required,0.0,0.167,Neutral
1,A ONG,P,2/4/2019,EPP,4,Fuel Pump 1 Pressure Low,Actions as required.,actions required,0.0,0.167,Neutral
2,A ONG,P,15/5/2019,EPP,4,Fuel Measure Uncertain,Actions as required.,actions required,0.0,0.167,Neutral
3,A ONG,P,15/5/2019,EPP,4,Engine Cut,Actions as required.,actions required,0.0,0.167,Neutral
4,A ONG,P,15/5/2019,EPP,4,GOC 1 Client Fail,Actions as required.,actions required,0.0,0.167,Neutral


In [3]:
df.head()
X=df['Specific comments']
y=df['Sentiment Term']

In [4]:
y.value_counts()

Positive    2325
Neutral     1354
Negative     610
Name: Sentiment Term, dtype: int64

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size =0.5, random_state=42)

In [6]:
vect = CountVectorizer()
#ngram_range=(1,2) min_df= max_df= 

In [7]:
# vect.get_feature_names()
X_train_dtm=vect.fit_transform(X_train)

In [8]:
X_test_dtm=vect.transform(X_test)

# Random Forest Classifier

In [9]:
rfc= RandomForestClassifier(n_estimators=200)
rfc.fit(X_train_dtm,y_train)
pred_rfc = rfc.predict(X_test_dtm)

In [10]:
metrics.accuracy_score(y_test, pred_rfc)

0.8722610722610723

In [11]:
metrics.confusion_matrix(y_test, pred_rfc)

array([[ 209,   73,   17],
       [  17,  591,   76],
       [   5,   86, 1071]], dtype=int64)

In [12]:
y_test.value_counts()

Positive    1162
Neutral      684
Negative     299
Name: Sentiment Term, dtype: int64

# Decision Tree Classifier

In [13]:
dtc= DecisionTreeClassifier()
dtc.fit(X_train_dtm,y_train)
pred_dtc = dtc.predict(X_test_dtm)

In [14]:
metrics.accuracy_score(y_test, pred_dtc)

0.8885780885780886

In [15]:
metrics.confusion_matrix(y_test, pred_dtc)

array([[ 240,   43,   16],
       [  34,  602,   48],
       [  17,   81, 1064]], dtype=int64)

In [16]:
y_test.value_counts()

Positive    1162
Neutral      684
Negative     299
Name: Sentiment Term, dtype: int64

In [17]:
min_leaf_options=list(range(1,11))
min_split_options=list(range(2,11))
#max_features_options=list(range(1,31))
#max_depth_options=list(range(1,31))
# weight_options = ['uniform','distance']
#type(max_depth_options)

In [18]:
param_grid1=dict(min_samples_leaf=min_leaf_options, min_samples_split=min_split_options)
# print (param_grid)

In [19]:
grid1=GridSearchCV(dtc,param_grid1,cv=10,scoring='accuracy',n_jobs=4)

In [20]:
X_grid=vect.fit_transform(X)
grid1.fit(X_grid,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [22]:
print(grid1.best_score_)
print(grid1.best_params_)

0.8925157379342504
{'min_samples_leaf': 1, 'min_samples_split': 3}


# Support Vector Classifier

In [23]:
svc= SVC()
svc.fit(X_train_dtm,y_train)
pred_svc = svc.predict(X_test_dtm)

In [24]:
metrics.accuracy_score(y_test, pred_svc)

0.5417249417249417

In [25]:
metrics.confusion_matrix(y_test, pred_svc)

array([[   0,    0,  299],
       [   0,    0,  684],
       [   0,    0, 1162]], dtype=int64)

In [26]:
y_test.value_counts()

Positive    1162
Neutral      684
Negative     299
Name: Sentiment Term, dtype: int64

# Logistic Regression

In [27]:
# logreg=LogisticRegression(max_iter=5000,penalty='l1',solver='liblinear')
logreg=LogisticRegression(max_iter=5000)
logreg.fit(X_train_dtm,y_train)
pred_logreg = logreg.predict(X_test_dtm)

In [28]:
metrics.accuracy_score(y_test, pred_logreg)

0.8741258741258742

In [29]:
metrics.confusion_matrix(y_test, pred_logreg)

array([[ 222,   67,   10],
       [  38,  560,   86],
       [   9,   60, 1093]], dtype=int64)

In [30]:
y_test.value_counts()

Positive    1162
Neutral      684
Negative     299
Name: Sentiment Term, dtype: int64

In [31]:
import numpy as np
penalty_options=['l1','l2']
solver_options=['liblinear','saga']

In [32]:
param_grid=dict(penalty=penalty_options,solver=solver_options)

In [33]:
grid = GridSearchCV(logreg,param_grid,cv=10,scoring='accuracy',n_jobs=4)

In [34]:
X_grid=vect.fit_transform(X)
grid.fit(X_grid,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [35]:
print(grid.best_score_)
print(grid.best_params_)

0.9020750757752389
{'penalty': 'l1', 'solver': 'liblinear'}
